# Competencia 2 NLP : Reconocimiento de entidades (NER) en Español

**Integrantes:** Alexander Cuevas - Jorge Gutiérrez - Benjamín Mellado - Felipe Urrutia

**Usuario del equipo en CodaLab:** *teamGalactico 🌌*

**Fecha límite de entrega 📆:** 24 de Junio a las 23:59 hrs.

**Tiempo estimado de dedicación:** En curso... ⏲️

**Link competencia:** [CC6205 Assignment 2 - Named Entity Recognition (NER) in Spanish](https://codalab.lisn.upsaclay.fr/competitions/5098?secret_key=09955d45-6210-4a35-a171-8050aa050855#learn_the_details)


## Introducción 🏷️

**REQUIREMENTS: Presentar brevemente el contexto, problema a resolver, incluyendo la formalización de la task (cómo son los inputs y outputs del problema) y los desafíos que ven al analizar el corpus entregado. (0.5 puntos)**

En este trabajo utilizaremos el Corpus de Listas de Espera propuesto por [1]. Este corpus esta constituido por derivaciones de diversas consultas de la lista de espera de los hospitales públicos chilenos. Por su naturaleza, cada una de estas derivaciones es un dato no-estructurado escrito en lenguage natural. Entre los resultados principales de [1] está otorgar un Corpus anotado usando texto clinico en español.

La tarea principal propuesta por [1] es un reconocimiento de entidades nombradas. Tambien conocida como NER, por sus siglas en ingles. Dicha tarea es una de las más populares en NLP. La cual consiste en identificar automaticamente piezas esenciales de informacion (entidades) en el texto. Tradicionalmente, la categoria de entidades a identificar eran nombres de personas, lugares y organizaciones. Actualmente, los tipos de categorias se han expandido a diversos dominios del conocimiento. En este trabajo, las entidades que estudiaremos viven en el contexto de texto clinico. 

[1] utiliza el esquema Unified Medical Language System (UMLS) para las categorias de entidades. Originalmente, [1] genera un Corpus con siete tipos de entidades: 

    (1) Finding, (2) Procedure, (3) Family Member, (4) Disease, (5) Body Part, (6) Medication y (7) Abbreviation. 
    
Sin embargo, en este trabajo solo estudiaremos cinco de ellas. Todas excepto (1) y (7). Las anotaciones siguen el esquema IOB2, estandar en NER. Esquema definido como sigue: 

>*Esquema IOB2*. Consiste en identificar cada token con una etiqueta que indica si pertenece o no a una entidad. Si un token **no** es parte de una entidad, se indican con $\texttt{O}$ (por la inicial en Ingles de *Fuera*). Sino, este necesariamente es un token de una entidad. Para indicar si el token es el inicio de una entidad, se utiliza el prefijo $\texttt{B-}$ (por la inicial en Ingles de *Comienzo*). En cambio, si el token es la continuacion de una entidad, se utiliza el prefijo $\texttt{I-}$ (por la inicial en Ingles de *Dentro*). Un ejemplo de este esquema, es el siguiente: 

\begin{align*}
&\texttt{PRESENTA}& &\texttt{FRACTURA}& &\texttt{CORONARIA}& &\texttt{COMPLICADA}& &\texttt{EN}& &\quad\text{(Texto)}\\
&\texttt{O}& &\texttt{B-Disease}& &\texttt{I-Disease}& &\texttt{I-Disease}& &\texttt{O}& &\quad\text{(IOB2)}
\end{align*}

Tal como se puede observar del ejemplo anterior, si se sigue el esquema de anotamiento IOB2 para la tarea de NER, el problema de reconocimiento de entidades nombradas se transforma en un problema de Sequence-Labeling. Tipo de problema que hemos discutido durante las ultimas unidades del curso. Entre los modelos estudiados estan HMM, CRF, MEMMS, CNN y RNN. En este trabajo, nos focalizaremos en solo modelos neuronales dentro de la familia de las Redes Neuronales Recurrentes (RNN, por sus siglas en Ingles).

Ahora bien, el conjunto de datos vienen entregados en un forma estandar para la tarea de NER, denominado ConLL. Este consiste en un archivo de texto con dos columnas y saltos de linea que separan cada secuencia. La primera columna, contiene los tokens de la secuencia. Mientras que la segunda columna, contiene las etiquetas IOB2 asociadas a cada token de la secuencia. Por otro lado, los datos vienen previamente separados en conjuntos de Entrenamiento, Desarrollo y Prueba. Estos conjuntos tiene tamaños X, Y y Z, resp. Ademas, la proporcion de etiquetas IOB2 son mayoritariamente $\texttt{O}$. Con una proporcion total de A, B y C, en cada conjunto resp. El segundo tipo de etiqueta IOB2 más frecuente es D, con proporciones E, F y G. Mientras que la etiqueta IOB2 menos frecuente es H, con proporiciones I, J, K. Esta diferencia considerable de proporciones de las etiquetas, se denomina desbalance de datos.

Entre las dificultades encontradas para está tarea están: (1) Desbalance de etiquetas IOB2, (2) [alguna otra encontrada del analisis exploratorio de datos] y (3) NER en contexto de texto clinico. Consideramos que dichas dificultades son desafiantes de resolver. Es por esto que es de nuestro interes lograr resolver satisfactoriamente esta tarea. En las secciones que siguen se encuentra detalladamente nuestra solución. 

**Referencias**

[1] Báez, P., Villena, F., Rojas, M., Durán, M., & Dunstan, J. (2020, November). The Chilean Waiting List Corpus: a new resource for clinical named entity recognition in Spanish. In Proceedings of the 3rd clinical natural language processing workshop (pp. 291-300). https://aclanthology.org/2020.clinicalnlp-1.32/

## Modelos 🤖

**REQUIREMENTS: Describir brevemente los modelos, métodos e hiperparámetros utilizados. (1.0 puntos)**

## Métricas de evaluación 📊

**REQUIREMENTS: Describir las métricas utilizadas en la evaluación indicando qué miden y cuál es su interpretación en este problema en particular. (0.5 puntos)**


En un problema de Sequence-labeling, la variable predicha por un modelo predictivo es un vector de etiquetas, donde cada etiqueta está asociada a un token en la secuencia del input. Una metrica usual para medir el desempeño de un modelo de este tipo determina TP, TN, FP y FN con cada etiqueta independientemente. Sin embargo, en la tarea de NER, el objetivo es reconocer entidades nombradas en el texto. Siendo las entidades, las predicciones importantes del modelo y no las etiquetas individuales en formato IOB2. Por esto, una metrica usual no cuantifica el desempeño del modelo para el problema original. Para ser más preciso, existen dos motivos, (1) las etiquetas importantes son aquellas asociadas a las entidades y (2) el modelo debe de reconocer exactamente tanto la ubicacion de la entidad como el tipo de entidad. Una metrica que si logra estas dos cosas, se denominda **metrica estricta**.

<!-- \begin{align*}
&\texttt{PRESENTA}& &\texttt{FRACTURA}& &\texttt{CORONARIA}& &\texttt{COMPLICADA}& &\texttt{EN}& &\quad\text{(Texto)}\\
&\texttt{O}& &\texttt{B-Disease}& &\texttt{I-Disease}& &\texttt{I-Disease}& &\texttt{O}& &\quad\text{(IOB2)}\\
&0& &1& &2& &3& &4& &\quad\text{(Indice)}
\end{align*} -->

En lo que sigue detallaremos las metricas estrictas utilizadas. Para formalizar su definicion, consideremos $D = (S, T)$ un conjunto de $n$ datos. Donde para cada secuencia $s \in S$, $t^s \in T$ corresponden a las etiquetas de $s$. Sin perdida de generalidad, supondremos que toda secuencia es de largo $m$. En la practica, $m$ corresponde al largo maximo de un secuencia dentro del conjunto de datos. Si una secuencia (y sus etiquetas) es de largo menor que $m$, se rellena con $\texttt{<pad>}$. Consideremos $P$ a las etiquetas predichas, i.e., $p^s \in P$ es la prediccion de la secuencia $s \in S$, comparable con las etiquetas reales $t^s \in T$. Naturalmente, $s_i$ es el i-esimo token de la secuencia $s$. Analogamente, $t^s_i$ para su etiqueta real y $p^s_i$ para la predicha. Se dira que $(l, i, j, c)$ es una entidad valida de tipo $c$ en las etiquetas $l$, si $i\leq j$, $l_i = \texttt{B-}c$, para cada $k \in \{i+1, ..., j\}$, $l_k = \texttt{I-}c$ y, si $j<m$, entonces $l_{j+1} \neq \texttt{I-}c$. Denotaremos por $E(l, c)$ al conjunto de las entidades validas de tipo $c$ en las etiquetas $l$. Adicionalmente, denotaremos por $E(L)$ al conjunto de todas las entidades validas segun $L$, i.e.,  $$E(L) = \bigcup \{E(l,  c): l \in L \text{ y $c$ es un tipo de entidad}\}$$
  
* *Precision*. Para un tipo de entidad fija (p.ej. Disease), la precision mide la proporcion de veces que una entidad predicha de tal tipo coincide con la entidad real sobre el total de veces que el modelo predice una entidad con el tipo deseado. En efecto, para una entidad de tipo $c$,

$$ \texttt{Precision}(c) = \frac{|\{ (l, i, j, d) \in E(P): d = c, \exists s \in S, l = p^s, (p^s, i, j, c) \in E(t^s, c)\}|}{|(l, i, j, d) \in E(P): d=c |}$$

* *Recall*. Para un tipo de entidad fija, el recall mide la proporcion de veces que una entidad predicha de tal tipo coincide con la entidad real sobre el total de entidades del tipo deseado. En efecto, para una entidad de tipo $c$,

$$ \texttt{Recall}(c) = \frac{|\{ (l, i, j, d) \in E(P): d = c, \exists s\in S, l=p^s, (p^s, i, j, c) \in E(t^s, c)\}|}{|(l, i, j, d) \in E(T): d=c |}$$

* *Micro F1 score*. **Recuerde hacer la distinción entre lo que sería una métrica de micro f1-score vs macro f1-score.** Para un tipo de entidad fija, la Precision y el Recall, entregan informacion distinta sobre las prediccion para dicho tipo de entidad. El primero, considera falsos-positivos, mientras que el segundo considera falsos-negativos. Una forma de condensar esta informacion en una unica metricas es con el promedio armonico entre Precision y Recall, denominado F1-score. Dicha metrica es condicional al tipo de entidad elegida. Para obtener un F1-score global (que considere todos los tipos de entidades), existen dos aproximaciones: (1) Micro F1-score y (2) Macro F1-score. En palabras simples, (1) es el Accuracy del modelo y (2) Un promedio de F1-score por tipo de entidad. Descartamos Macro F1-score dado que nuestros datos son desbalancedados. Por lo tanto, cada F1-score por clase no son estadisticamente comparables. Nos quedaremos con Micro F1-score, ya que traduce globalmente el desempeño del modelo. Para computar esta metrica, se utiliza la tecnica estricta definida anteriormente. En efecto, 

$$ \texttt{Micro-F1-score} = \frac{|\{ (l, i, j, d) \in E(P): \exists s\in S, l=p^s, (p^s, i, j, d) \in E(t^s, d)\}|}{|S|}$$



## Diseño experimental ✏️

**REQUIREMENTS: Esta es una de las secciones más importantes del reporte. Deben describir minuciosamente los experimentos que realizarán en la siguiente sección. Describir las variables de control que manejarán, algunos ejemplos pueden ser: Los hiperparámetros de los modelos, tipo de embeddings utilizados, tipos de arquitecturas. Ser claros con el conjunto de hiperparámetros que probarán, la decisión en las funciones de optimización, función de pérdida, regulación, etc. Básicamente explicar qué es lo que veremos en la siguiente sección. (1 punto)**

## Experimentos 🧪

**REQUIREMENTS: Reportar todos sus experimentos y código en esta sección. Comparar los resultados obtenidos utilizando diferentes modelos. ¡Es vital haber realizado varios experimentos para sacar una buena nota! (2.0 puntos)**

## Conclusiones 💡

**REQUIREMENTS: Discutir resultados, proponer trabajo futuro. (1 punto)**